# 2DEG mobility plottings (300K)

# 1. Settings

## 1.1 Import modules

In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import colors, cm
import matplotlib.ticker as ticker

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
from pathlib import Path
mobilitypy_tutorial_path = Path().absolute()
sys.path.insert(0, os.path.join(mobilitypy_tutorial_path, '../'))
from mobilitypy import Plottings, PlotQuasi3DFuns

In [4]:
save_figure_dir = os.path.join(mobilitypy_tutorial_path,'../../OUTPUTS/FIGS')
savefigure = True
fig_save_dpi = 75
color_map_plot = 'viridis'
print_align_space = 20

In [5]:
folder_2deg_data_parent = '/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/NextnanoSimulations/DATAs/AlGaN_HEMT/AlGaN_AlGaN_AlN_HEMT/nnp/'
folder_2deg_density_file = os.path.join(folder_2deg_data_parent, '2DEHG_density/OneDim', 'sim_post_process_data.xlsx')
folder_2deg_mobility_file = os.path.join(folder_2deg_data_parent, '2DEHG_mobility/OneDim', 'sim_post_process_data.xlsx')

## 1.2 Set physical constants

In [6]:
# n_2d => in nm^-2
# rms_roughness => nm^-1
# corr_len => nm^-1
# n_dis => nm^-2
# f_dis => unit less

In [7]:
plot_mobilities = ['IFR', 'DIS', 'AD', 'AP', 'POP', 'TOT', 'LFOM', 'LFOMnorm', '2DEG']
plots_list_individual = ['TOT', 'LFOM', 'LFOMnorm', '2DEG', 'R']
len_plots = len(plot_mobilities)
rescale_2deg_fact=1e13

x_label_text = 'Al composition barrier'
x_label_text_2d = 'Barrier thickness (nm)'
x_p_label_text = 'Al composition channel'
y_label_text = 'Barrier thickness (nm)'
z_label = {'General': r'$\mu$ ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)', 
           'TOT': r'$\mu$ ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)',
           'LFOM': r'LFOM (MW/cm$^2$)',
           'LFOMnorm': r'LFOM$_\mathrm{AlGaN}$/LFOM$_\mathrm{highest}$', 
           'LFOMnorm_2': r'LFOM$_\mathrm{AlGaN}$/LFOM$_\mathrm{GaN}$',
           '2DEG': r'2DEG ($\mathrm{10}^{13}\mathrm{cm}^{-2}$)', 
           'R': r' R ($\Omega /\boxdot$)'} #Sheet resistance

In [8]:
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize':20,
          'xtick.major.width':2,
          'xtick.major.size':5,
          'xtick.minor.width':2,
          'xtick.minor.size':3,
          'ytick.labelsize': 20,
          'ytick.major.width':2,
          'ytick.major.size':5,
          'ytick.minor.width':2,
          'ytick.minor.size':3,
          'errorbar.capsize':2}
plt.rcParams.update(params)
plt.rc('font', size=20)

# 2. Read mobilities, FOMs from saved database

## 2.1 Al0.25Ga0.75N(25nm)/GaN(305nm) HEMT LFOM for normalization

In [9]:
mobility_ref_dff = pd.read_excel(folder_2deg_mobility_file, index_col=0, sheet_name='ref_Al25Ga75N_GaN') 

In [10]:
mobility_ref_df = mobility_ref_dff[mobility_ref_dff['T'] == 300].copy() # Reference data at 300 K
ref_LFOM = mobility_ref_df['LFOM'].iloc[0]

Conclusion: 2DEG density negligible depend on temperature

In [11]:
print(f'{"- Reference sample":<{print_align_space}}: Al0.25Ga0.75N(25nm)/GaN(305nm)')
print(f'{"-- 2DEG density":<{print_align_space}}: {mobility_ref_df['2DEG_device'].iloc[0]:.5e} cm^-2')
print(f'{"-- 2DEG mobility":<{print_align_space}}: {mobility_ref_df['TOT'].iloc[0]:.5e} cm^2.V^-1.s^-1')
print(f'{"-- LFOM":<{print_align_space}}: {ref_LFOM:.5e} MW/cm^2')

- Reference sample  : Al0.25Ga0.75N(25nm)/GaN(305nm)
-- 2DEG density     : 6.25000e+12 cm^-2
-- 2DEG mobility    : 1.55582e+03 cm^2.V^-1.s^-1
-- LFOM             : 2.21367e+04 MW/cm^2


## 2.2 AlGaN/AlGaN mobilities and figure-of-merit

In [12]:
mobility_dff = pd.read_excel(folder_2deg_mobility_file, index_col=0, sheet_name='AlGaN_AlGaN') 
# Collect data for 300 K temperature only
mobility_dff_300k = mobility_dff[mobility_dff['T'] == 300].copy()

In [13]:
mobility_dff_300k['2DEG'] =  mobility_dff_300k['2DEG_device']/rescale_2deg_fact
mobility_dff_300k['2DHG'] =  mobility_dff_300k['2DHG_device']/rescale_2deg_fact
mobility_dff_300k['LFOMnorm'] = mobility_dff_300k['LFOM']/ref_LFOM

In [14]:
mobility_df_300k = mobility_dff_300k.copy()

# Put 10^10 for large values and np.nan for values < 1
clean_up_mobilities = ['IFR', 'DIS', 'AD', 'AP', 'POP', 'TOT']
for which_column in clean_up_mobilities:
    mobility_df_300k.loc[mobility_df_300k[which_column] <1, which_column] = np.nan 

# Put 10^10 for large values and np.nan for values < 1
for which_column in ['2DEG', 'LFOM', 'LFOMnorm']:
    mobility_df_300k.loc[mobility_df_300k[which_column] <1e-3, which_column] = np.nan 

## 2.4 Plot AlGaN/AlGaN HEMT mobilities

In [15]:
tick_multiplicator = [0.1, 0.05, 0.1, 0.05]

In [16]:
df_cap_thickness_group = mobility_df_300k.groupby(['ThicknessAlGaNBarrier']) # Group by thickness

### 2.4.1 All contributions together

In [17]:
lpltq3d = PlotQuasi3DFuns(save_figure_dir=f'{save_figure_dir}/2DEG_mobilities/Together')

In [18]:
fig_ncols, fig_nrows = 3, 3
for name, group in df_cap_thickness_group:
    barrier_thickness = f'{name[0]:.2f}'
    print(f'- Plotting barrier thickness = {barrier_thickness} nm')
    fig, axs = plt.subplots(fig_ncols, fig_nrows, figsize=(16.,13.2), 
                            sharey=True,constrained_layout=True)    
    for ii in range(fig_ncols*fig_nrows):
        if ii < len(plot_mobilities):
            which_mobility = plot_mobilities[ii]
        else:
            axs[ii//3][ii%3].axis("off") 
            continue
        print(f'\t-- Plotting {which_mobility}...')
        xx, yy, zz = group['AlContentChannel'], group['AlContentBarrier'], group[which_mobility]
        vmax = min(mobility_df_300k[which_mobility].max(numeric_only=True), 1e10)
        vmin = mobility_df_300k[which_mobility].min(numeric_only=True) if which_mobility in ['2DEG', 'LFOM', 'LFOMnorm'] else 10
        #vmin, vmax = zz.min()+1e-20, zz.max()
        norm = colors.Normalize(vmin=vmin, vmax=vmax) if which_mobility in ['LFOMnorm','2DEG'] else colors.LogNorm(vmin=vmin, vmax=vmax)
        cbar_mapable = cm.ScalarMappable(norm=norm, cmap='viridis')
        x_label_text_ = x_p_label_text if ii-6>=0 else ''
        y_label_text_ = x_label_text if ii%3==0 else ''
        z_label_ = ''
        if ii%3-2==0: z_label_ = z_label['General']
        if which_mobility in ['2DEG', 'LFOM']: z_label_= z_label[which_mobility]
        if which_mobility == 'LFOMnorm': z_label_= z_label[f'{which_mobility}_2'] 
        

        ##### Plot composition map 
        fig, _ = lpltq3d.Plotq3D(xx,yy,zz, fig=fig, ax=axs[ii//3][ii%3],
                                 xmin=0.475, xmax=0.975, ymin=0.525, ymax=1.025,
                                 x_label=x_label_text_, y_label=y_label_text_,
                                 interpolation_method='linear',
                                 interpolation_points = 20,
                                 tick_multiplicator=tick_multiplicator,
                                 title_label=which_mobility,
                                 cbar_mappable=cbar_mapable, norm=norm,
                                 show_contour_lines=False, marker='s', marker_size=24**2,
                                 cbar_text=z_label_,show_colorbar=True,
                                 plot_controur=0, plot_scatter=1,
                                 savefigure=False, show_plot=False)
        plt.suptitle(f"Barrier thickness = {barrier_thickness} nm", size=24)
        if ii%3 != 0: axs[ii//3][ii%3].get_yaxis().set_visible(False)
    lpltq3d.save_figure(f'Barrier_{barrier_thickness}.png', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)
    print('- Done', flush=True)
    #break

- Plotting barrier thickness = 5.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 10.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 15.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 20.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 25.00 nm
	-- Plotting IFR...
	-- Plotting 

### 2.4.2 Individual plots

In [19]:
lpltq3d = PlotQuasi3DFuns(save_figure_dir=f'{save_figure_dir}/2DEG_mobilities/Individual')

In [20]:
for name, group in df_cap_thickness_group:
    #if name[0] < 49: continue
    barrier_thickness = f'{name[0]:.2f}'
    print(f'- Plotting barrier thickness = {barrier_thickness} nm')
    for which_mobility in plots_list_individual:
        print(f'\t-- Plotting {which_mobility}...')
        xx, yy, zz = group['AlContentChannel'], group['AlContentBarrier'], group[which_mobility]
        vmax = min(mobility_df_300k[which_mobility].max(numeric_only=True), 1e10)
        vmin = mobility_df_300k[which_mobility].min(numeric_only=True) if which_mobility in ['2DEG', 'LFOMnorm'] else 10 
        if which_mobility == 'R': vmin = mobility_df_300k[which_mobility].min(numeric_only=True)
        norm = colors.Normalize(vmin=vmin, vmax=vmax) if which_mobility in ['2DEG', 'LFOMnorm'] else colors.LogNorm(vmin=vmin, vmax=vmax)
        cbar_mapable = cm.ScalarMappable(norm=norm, cmap=color_map_plot)
        z_label_= z_label[which_mobility]
        if which_mobility == 'LFOMnorm': z_label_= z_label[f'{which_mobility}_2'] 
    
        ##### Plot composition map 
        _ = lpltq3d.Plotq3D(xx, yy, zz, x_label=x_p_label_text, y_label=x_label_text,
                            xmin=0.475, xmax=0.975, ymin=0.525, ymax=1.025,
                            interpolation_method='linear', interpolation_points = 20,
                            tick_multiplicator=tick_multiplicator, norm=norm,color_map=color_map_plot,
                            cbar_mappable=cbar_mapable, show_contour_lines=1,
                            marker='s', marker_size=38**2, cbar_text=z_label_,
                            show_colorbar=True, plot_controur=0, plot_scatter=1,
                            save_file_name=f'Barrier_{barrier_thickness}_{which_mobility}.png',
                            savefigure=savefigure, show_plot=False, dpi=fig_save_dpi)
    print('- Done', flush=True)
    #break

- Plotting barrier thickness = 5.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 10.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 15.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 20.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 25.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 30.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
	-- Plotting R...
- Done
- Plotting barrier thickness = 35.00 nm
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plot

In [21]:
plt.close('all')

## 2.5 Plots for AlN/AlGaN

In [22]:
plt2deg = Plottings(save_figure_dir=f'{save_figure_dir}/Others')

### 2.5.1 Mobility plot

In [23]:
x_label = 'Barrier thickness (nm)'
y_label = z_label['TOT'] #r'Electron mobility ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)'
save_file_name_ = 'mu_Tb_AlN_AlGaN.png'
tick_multiplicator=[10, 5, 50, 25]

In [24]:
mobility_df_300k_ = mobility_dff_300k[mobility_dff_300k['AlContentBarrier']>0.99]
df_channel_comp_group = mobility_df_300k_.groupby(['AlContentChannel']) # Group by channel composition

fig, axs = plt.subplots(constrained_layout=True)
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['TOT']
    axs.plot(xx, zz, 'o-',label=f'{name[0]:.2f}')
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left', title='Al-content') 
axs.set_xlim(5,50)
plt2deg.set_tickers(axs, tick_multiplicator)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

In [25]:
save_file_name_ = 'channel_comp_legends.png'
# Now create a new image with legends only
# adjust the figure size as necessary
fig_leg = plt.figure(figsize=(1,1))
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*axs.get_legend_handles_labels(), ncol=5, loc='center')
# hide the axes frame and the x/y labels
ax_leg.axis('off')
#ax_leg.set_frame_on(False)
plt2deg.save_figure(save_file_name_, fig=fig_leg, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

In [26]:
fig, axs = plt.subplots(constrained_layout=True) #figsize=(9.5,6)
for name, group in df_channel_comp_group:
    if name[0]< 0.74 or name[0]> 0.91: 
        axs.plot([None], [None], label=None)
        continue
    xx, zz = group['ThicknessAlGaNBarrier'],  group['TOT']
    axs.plot(xx, zz, 'o-',label=f'{name[0]:.2f}')
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
plt2deg.save_figure(f'zoom_{save_file_name_}', fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.2 Individual mobility contributions for AlN/Al0.9Ga0.1N

In [27]:
Al90Ga10N_mu_df_300k_ = mobility_df_300k_[(mobility_df_300k_['AlContentChannel']>0.89) & (mobility_df_300k_['AlContentChannel']<0.91)]

In [28]:
fig, axs = plt.subplots(figsize=(9.5,6),constrained_layout=True)
for name in plot_mobilities[:-3]:
    axs.plot(Al90Ga10N_mu_df_300k_['ThicknessAlGaNBarrier'], Al90Ga10N_mu_df_300k_[name], 'o-',label=f'{name}')
axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
axs.set_ylim(100, 100000)
axs.set_xlim(5,50)
plt2deg.set_tickers(axs, [10,5,None, None])
plt2deg.save_figure('mu_contibs_Tb_AlN_Al0.9Ga0.1N.png', fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.3 FOM plot

In [29]:
x_label = x_label_text_2d
y_label = z_label['LFOMnorm_2']
save_file_name_ = 'lfom_Tb_AlN_AlGaN.png'
tick_multiplicator=[10, 5, 0.5, 0.25]

In [30]:
fig, axs = plt.subplots(constrained_layout=True)
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['LFOMnorm']
    axs.plot(xx, zz, 'o-',label=f'{name[0]:.2f}')
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
axs.axhline(y=1.0, c='k', ls='--')
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

In [31]:
fig, axs = plt.subplots(constrained_layout=True)
for name, group in df_channel_comp_group:
    if name[0]< 0.74 or name[0]> 0.91: 
        axs.plot([None], [None])
        continue
    xx, zz = group['ThicknessAlGaNBarrier'],  group['LFOMnorm']
    axs.plot(xx, zz, 'o-',label=f'{name[0]:.2f}')
#axs.set_yscale('log')
axs.axhline(y=1.0, c='k', ls='--')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
plt2deg.save_figure(f'zoom_{save_file_name_}', fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.4 2DEG density plot

In [32]:
x_label = x_label_text_2d
y_label = z_label['2DEG']
save_file_name_ = 'deg_Tb_AlN_AlGaN.png'
tick_multiplicator=[10, 5, None, None]

In [33]:
fig, axs = plt.subplots(figsize=(8,6),constrained_layout=True)
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['2DEG']
    axs.plot(xx, zz, 'o-',label=f'{name[0]:.2f}')
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
axs.axhline(y=1.0, c='k', ls='--')
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
axs.set_xlim(5,50)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.4 Sheet resitance plot

In [34]:
x_label = x_label_text_2d
y_label = z_label['R']
save_file_name_ = 'R_Tb_AlN_AlGaN.png'
tick_multiplicator=[10, 5, None, None]

In [35]:
fig, axs = plt.subplots(figsize=(8,6),constrained_layout=True)
for name, group in df_channel_comp_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['R'].copy()
    zz[zz>1e8] = np.nan
    axs.plot(xx, zz, 'o-',label=f'{name[0]:.2f}')
axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
#axs.axhline(y=1.0, c='k', ls='--')
#axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
#plt2deg.set_tickers(axs, tick_multiplicator)
#axs.set_xlim(5,50)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

### 2.5.5 Mobility contributions for AlN(50nm)/AlGaN

In [36]:
aln_algan_300k = mobility_dff_300k[(mobility_dff_300k['AlContentBarrier']>0.99) & (mobility_dff_300k['ThicknessAlGaNBarrier']>49)]
aln_algan_dff = aln_algan_300k[['AlContentChannel','IFR','AD','DIS','DP','PE','AP','POP','TOT']].copy()
aln_algan_dff.rename(columns={'AlContentChannel':'comp'}, inplace=True)

x_label = x_p_label_text
y_label = z_label['TOT']
save_file_name_ = 'mu_AlN50_AlGaN.png'
tick_multiplicator=[0.1, 0.05, None, None]

In [37]:
# Mobility contributions at 300K
save_file_name_ = 'Mobility_contribs_AlN50AlGaN_300K.png'
x_label = 'Al composition'
y_label = z_label['TOT']
fig, ax,_ = plt2deg.plot_2d_carrier_mobilities(aln_algan_dff, save_file_name=save_file_name_,
                                               ymin=5e1, ymax=2e5, xmax=0.9, xmin=0.5, y_scale_log=True, 
                                               annotate_pos=(1,1), show_right_ticks=True,
                                               mode='2d_carrier_mobility', yaxis_label=y_label, xaxis_label=x_label,
                                               color=None, color_map='viridis', savefig=0, show_plot=False)
plt2deg.set_tickers(ax, tick_multiplicator)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

In [38]:
fig, ax = plt.subplots(figsize=(9,6), constrained_layout=True)
XX = np.array(aln_algan_300k['AlContentChannel'], dtype=float)
ax.plot(XX, aln_algan_300k['2DEG_device']/rescale_2deg_fact, 'o-', c='k')
ax.set_xlim(0.5,0.9)
ax.set_xlabel(x_p_label_text)
ax.set_ylabel(z_label['2DEG'])
plt2deg.set_tickers(ax, tick_multiplicator)

# Plotting mobility
ax2 = ax.twinx() 
ax2.plot(XX, aln_algan_300k['TOT'], 'o-', c='r')

# Plotting bandgap
bandgap_ = (6.20*XX + 3.43*(1-XX) + 0.7*XX*(1-XX))**5 
ax2.plot(XX, bandgap_, 'o-', c='m')

ax2.set_ylabel(z_label['TOT'], c='r')
ax2.set_yscale('log')
ax2.set_ylim(ymin=70, ymax=1e4)
ax2.annotate(r'E$_\mathrm{g}^{5}$', xy=(XX[3], bandgap_[3]+1e3), color='m')

# Plotting sheet resistance
R_300k = np.array(aln_algan_300k['R'], dtype=float) #/20000 #1/20000 is to rescale
ax2.plot(XX, R_300k, 'o-', c='g')
ax2.annotate(r'R', xy=(XX[3], R_300k[3]-1e3), color='g')

# Plotting LFOM
LFOM_300k = np.array(aln_algan_300k['LFOM'], dtype=float)/ref_LFOM 
ax.plot(XX, LFOM_300k, 'o-', c='b')
ax.annotate(r'LFOM$_{\mathrm{norm}}$', xy=(XX[3], LFOM_300k[3]-0.2), color='b')

plt2deg.save_figure(f'AlN50AlGaN_300K_mu_FOM.png', savefig=savefigure, show_plot=True,
                    fig=fig, CountFig=None, dpi=fig_save_dpi)